In [53]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
#from qiskit_aer.library import save_statevector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import numpy as np
from qiskit.quantum_info import Statevector

In [42]:
np.random.seed(42)

# Defining relevant methods.
Note that this implementation, does not use ancillary bits to detect which qubit is bit/phase flipped , i.e without stabiliser measurement(2 qubits per circuit saved), but rather just detects and correct the main qubit.

In [43]:
def bit_flip_encode(qc,qubits):
    "encodes the bit flip code, 2nd 3rd index of qubits are the added physical qubits"
    qc.cx(qubits[0], qubits[1])
    qc.cx(qubits[0], qubits[2])
def phase_flip_encode(qc,qubits):
    "encodes the phase flip code, 2nd 3rd index of qubits are the added physical qubits"
    bit_flip_encode(qc,qubits)
    qc.h(qubits)
def bit_flip_detect_correct(qc,qubits):
    "detects and corrects the bit flip on first qubit"
    qc.cx(qubits[0], qubits[1])
    qc.cx(qubits[0], qubits[2])
    qc.ccx(qubits[1], qubits[2], qubits[0])

def phase_flip_detect_correct(qc,qubits):
    "detects and corrects the phase flip on first qubit"
    qc.h(qubits)
    qc.cx(qubits[0], qubits[1])
    qc.cx(qubits[0], qubits[2])
    qc.ccx(qubits[1], qubits[2], qubits[0])

# Bit Flip detection and correction circuit

In [ ]:
# Bit flip code
qr1 = QuantumRegister(3, 'q')
qc1 = QuantumCircuit(qr1)

#preparing random qubit state
qc1.rx(np.pi/np.random.randint(2,10),qr1[0])
qc1.ry(np.pi/np.random.randint(2,10),qr1[0])
qc1.rz(np.pi/np.random.randint(2,10),qr1[0])

# encoding for bit flip detection and correction
qc1.save_statevector(label="initial state")
bit_flip_encode(qc1,qr1)
qc1.save_statevector(label="after encoding")

# introducing error on the anyone qubit
qc1.x(qr1[np.random.randint(0,3)])
qc1.save_statevector(label="after error")

# detection and correction
bit_flip_detect_correct(qc1,qr1)
qc1.save_statevector(label="after correction")

qc1.draw("text")


┌─────────┐┌─────────┐┌─────────┐ initial state            after encoding »
q_0: ┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/9) ├───────░─────────■────■─────────░────────»
     └─────────┘└─────────┘└─────────┘       ░       ┌─┴─┐  │         ░        »
q_1: ────────────────────────────────────────░───────┤ X ├──┼─────────░────────»
                                             ░       └───┘┌─┴─┐       ░        »
q_2: ────────────────────────────────────────░────────────┤ X ├───────░────────»
                                             ░            └───┘       ░        »
c: 4/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«     ┌───┐ after error           ┌───┐ after correction 
«q_0: ┤ X ├──────░────────■────■──┤ X ├────────░─────────
«     └───┘      ░      ┌─┴─┐  │  └─┬─┘        ░         
«q_1: ───────────░──────┤ X ├──┼────■──────────░─────────
«                ░      └───┘┌─┴─┐  │          ░         
«q_2: ───────────░───────────┤ X ├──■──────────░─────────
«                ░           └───┘             ░         
«c: 4/═══════════════════════════════════════════════════
«

In [ ]:
sim = AerSimulator(method='statevector')
pm = generate_preset_pass_manager(backend=sim)
qc1 = pm.run(qc1)
res1 = sim.run(qc1).result().data()

In [ ]:
res1["initial state"].draw(output="latex")   # main qubit is rightmost

<IPython.core.display.Latex object>

In [ ]:
res1["after encoding"].draw(output="latex")

<IPython.core.display.Latex object>

In [ ]:
res1["after error"].draw(output="latex")

<IPython.core.display.Latex object>

In [ ]:
res1["after correction"].draw(output="latex")

<IPython.core.display.Latex object>

# Phase flip detection and correction

In [74]:
# phase flip detection and correction
qr2 = QuantumRegister(3, 'q')
qc2 = QuantumCircuit(qr2)

#preparing random qubit state
qc2.rx(np.pi/np.random.randint(2,10),qr2[0])
qc2.ry(np.pi/np.random.randint(2,10),qr2[0])
qc2.rz(np.pi/np.random.randint(2,10),qr2[0])

# encoding for phase flip detection and correction
qc2.save_statevector(label="initial state")
phase_flip_encode(qc2,qr2)
qc2.save_statevector(label="after encoding")

# introducing error on the anyone qubit
qc2.z(qr2[np.random.randint(0,3)])
qc2.save_statevector(label="after error")

# detection and correction
phase_flip_detect_correct(qc2,qr2)
qc2.save_statevector(label="after correction")

qc2.draw("text")

┌─────────┐┌─────────┐┌─────────┐ initial state           ┌───┐»
q_0: ┤ Rx(π/5) ├┤ Ry(π/9) ├┤ Rz(π/9) ├───────░─────────■────■──┤ H ├»
     └─────────┘└─────────┘└─────────┘       ░       ┌─┴─┐  │  ├───┤»
q_1: ────────────────────────────────────────░───────┤ X ├──┼──┤ H ├»
                                             ░       └───┘┌─┴─┐├───┤»
q_2: ────────────────────────────────────────░────────────┤ X ├┤ H ├»
                                             ░            └───┘└───┘»
«      after encoding       after error ┌───┐          ┌───┐ after correction 
«q_0: ───────░───────────────────░──────┤ H ├──■────■──┤ X ├────────░─────────
«            ░                   ░      ├───┤┌─┴─┐  │  └─┬─┘        ░         
«q_1: ───────░───────────────────░──────┤ H ├┤ X ├──┼────■──────────░─────────
«            ░        ┌───┐      ░      ├───┤└───┘┌─┴─┐  │          ░         
«q_2: ───────░────────┤ Z ├──────░──────┤ H ├─────┤ X ├──■──────────░─────────
«            ░        └───┘      ░      └───┘     └───┘             ░

In [76]:
sim = AerSimulator(method='statevector')
pm = generate_preset_pass_manager(backend=sim)
qc2 = pm.run(qc2)
res2 = sim.run(qc2).result().data()

In [78]:
res2["initial state"].draw(output="latex")   # main qubit is rightmost

<IPython.core.display.Latex object>

In [79]:
res2["after encoding"].draw(output="latex")

<IPython.core.display.Latex object>

In [80]:
res2["after error"].draw(output="latex")

<IPython.core.display.Latex object>

In [81]:
res2["after correction"].draw(output="latex")

<IPython.core.display.Latex object>

# Answers to some theoretical questions
## 1: Limitations of bit-flip code
> - It can only correct a single type error (either bit or phase flip) at a time
> - It can only correct single qubit bit/phase flip, as it's based on majority voting and can't be used if error probabilites are >high enough that chances of errors on more than one qubits is not small.
> - To figure out the syndrome, ancilla qubits are required (although just correction can be done without it), which adds to >complexity and more point of failure.

## 2: Can you interchange the bit and phase flip detection and correction modules in shor's code
> - if the stabilizer measurements are being done, then stabilizer's must commute to get the same behaviour upon interchanging bit >and phase flip detection and correction modules.

## 3: Why is fault tolerance is important?
> - To prevent any error propagation, due to any faulty gate (coherent errors) and interaction with environment (incoherent errors).
> - Essential for practical quantum computers where physical qubits are noisy, and error correction cycles are frequent.
> - To ensure logical error rates scale favourably with code distance